# MNIST

On this noteboook, we try to add and preprocess our own dataset manually and make our own config


https://elitedatascience.com/keras-tutorial-deep-learning-in-python

https://medium.com/@mjbhobe/mnist-digits-classification-with-keras-ed6c2374bd0e

In [1]:
# imports
import matplotlib.pyplot as plt
from loguru import logger
import tensorflow as tf

import sys
sys.path.append('../../')
import main
import multi_partner_learning
import scenario
import utils
from dataset import Dataset

from pathlib import Path


import numpy as np
 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

Using TensorFlow backend.


# Create custom scenario from mandatory parametters

In [2]:
scenario_params = {
    'partners_count': 3,
    'amounts_per_partner': [0.2, 0.5, 0.3],
}

# Set values for optinal parametters

In [3]:
scenario_params['epoch_count'] = 10
scenario_params['minibatch_count'] = 3

#### Every other parametter will be set to its default value

In [4]:
experiment_path = Path(r"C:\GitHub\distributed-learning-contributivity\experiments\trash")

In [5]:
current_scenario = scenario.Scenario(
        scenario_params,
        experiment_path
    )

2020-08-07 12:51:55.657 | DEBUG    | scenario:__init__:52 - Dataset selected: mnist
2020-08-07 12:51:55.658 | DEBUG    | scenario:__init__:87 - Computation use the full dataset for scenario #1
2020-08-07 12:51:55.736 | INFO     | scenario:__init__:279 - ### Description of data scenario configured:
2020-08-07 12:51:55.737 | INFO     | scenario:__init__:280 -    Number of partners defined: 3
2020-08-07 12:51:55.737 | INFO     | scenario:__init__:281 -    Data distribution scenario chosen: random
2020-08-07 12:51:55.738 | INFO     | scenario:__init__:282 -    Multi-partner learning approach: fedavg
2020-08-07 12:51:55.738 | INFO     | scenario:__init__:283 -    Weighting option: uniform
2020-08-07 12:51:55.739 | INFO     | scenario:__init__:284 -    Iterations parameters: 10 epochs > 3 mini-batches > 8 gradient updates per pass
2020-08-07 12:51:55.739 | INFO     | scenario:__init__:290 - ### Data loaded: mnist
2020-08-07 12:51:55.739 | INFO     | scenario:__init__:291 -    48000 train dat

# Create Data Set

In [6]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()


X_train = X_train.reshape(X_train.shape[0],  28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

input_shape = (28, 28, 1)
num_classes = 10

# Create Preprocessing function

In [7]:
def preprocess_dataset_labels(y):
    y = np_utils.to_categorical(y, 10)
    return y

# Create Model

In [8]:
def generate_new_model_for_dataset():
    model = Sequential()
    # add Convolutional layers
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))    
    model.add(Flatten())
    # Densely connected layers
    model.add(Dense(128, activation='relu'))
    # output layer
    model.add(Dense(num_classes, activation='softmax'))
    # compile with adam optimizer & categorical_crossentropy loss function
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Assignate dataset to scenario

In [9]:
current_scenario.dataset = Dataset(
    "my_dataset",
    X_train,
    X_test,
    y_train,
    y_test,
    input_shape,
    num_classes,
    preprocess_dataset_labels,
    generate_new_model_for_dataset
)

In [10]:
# Check Scenario name
print(current_scenario.dataset.name)

my_dataset


# Split train and validation sets

In [11]:
current_scenario.dataset.train_val_split()

# Legacy 

In [12]:
current_scenario.partners_list = []

# Run scenario

In [13]:
main.run_scenario(current_scenario)

2020-08-07 12:51:56.381 | INFO     | scenario:split_data:534 - ### Splitting data among partners:
2020-08-07 12:51:56.382 | INFO     | scenario:split_data:535 -    Simple split performed.
2020-08-07 12:51:56.382 | INFO     | scenario:split_data:536 -    Nb of samples split amongst partners: 38880
2020-08-07 12:51:56.383 | INFO     | scenario:split_data:538 -    Partner #0: 7776 samples with labels [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2020-08-07 12:51:56.383 | INFO     | scenario:split_data:538 -    Partner #1: 19440 samples with labels [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2020-08-07 12:51:56.384 | INFO     | scenario:split_data:538 -    Partner #2: 11664 samples with labels [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2020-08-07 12:51:56.531 | DEBUG    | scenario:compute_batch_sizes:582 -    Compute batch sizes, partner #0: 324
2020-08-07 12:51:56.532 | DEBUG    | scenario:compute_batch_sizes:582 -    Compute batch sizes, partner #1: 810
2020-08-07 12:51:56.532 | DEBUG    | scenario:compute_batch_sizes:582 -   

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_857]

Function call stack:
keras_scratch_graph


# Results

In [ ]:
df_results = current_scenario.to_dataframe()
df_results["random_state"] = 1
df_results["scenario_id"] = 1
print(df_results.columns)

In [ ]:
print(df_results.mpl_test_score)

In [ ]:
print(current_scenario.mpl)

# Extract model 

In [ ]:
model = current_scenario.mpl.get_model()

In [ ]:
model.evaluate(X_test, preprocess_dataset_labels(y_test))

Seems nice as a federated learning result